# Flag Project!!

## TODO
- Translation logic and methods
- Country names -> Language -> Language Codes
- Clean and modularize more

### Installing packages
 googletrans package for translation methods  
 pycountry to help match country name and language code. Hopefully.

In [50]:
!pip install -r requirements.txt

In [51]:
from email.mime.text import MIMEText
from googletrans import Translator
import requests
from requests import get
import re
import sys
import os
import ast
import re
import urllib
import smtplib
import base64
from email.mime.text import MIMEText
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from requests import HTTPError
from googletrans import Translator
from tqdm.notebook import tqdm

### Setting up URL

In [5]:
url = "https://www.gov.br/mre/pt-br/assuntos/Embaixadas-Consulados-Missoes/de-outros-paises-no-brasil"

### Getting Embassies Webpage

In [6]:
print("Crawling URL %s" % url)
try:
    response = requests.get(url)
    print("ok!")
except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError) as ex:
    raise ex

Crawling URL https://www.gov.br/mre/pt-br/assuntos/Embaixadas-Consulados-Missoes/de-outros-paises-no-brasil
ok!


In [7]:
#This reges matches all Embassies and Emails
emails_names = (re.findall(r"[a-z0-9+_][a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z0-9\.\-+_]+|Embaixada d[a-z0-9^A-zÀ-ú\-+_\s]+|Consulado[\-\s][a-z0-9^A-zÀ-ú\-+_\s]+", response.text, re.I))

### Cleaning email and embassies name list.

In [8]:
#removes duplicates names and emails
mylist = list(dict.fromkeys(emails_names))

In [9]:
def create_diplomatic_mission_dict(email_list):
    diplomatic_missions = {}
    current_mission = ""
    used_emails = set()

    for item in email_list:
        if "Embaixada" in item or "Consulado" in item:
            current_mission = item
            if current_mission not in diplomatic_missions:
                diplomatic_missions[current_mission] = []
        elif "@" in item:
            email = item.strip()
            if email not in used_emails:
                diplomatic_missions[current_mission].append(email)
                used_emails.add(email)

    return diplomatic_missions

In [11]:
emails_dict = create_diplomatic_mission_dict(emails_names[2:])

In [12]:
# emails_dict

## Setting up translation, email config and mailing methods

#### Country:Code Dict

In [14]:
country_code = {
    "Áustria": ["de"],
    "África do Sul": ["af", "en"],
    "Afeganistão": ["fa", "ps", "uz", "tk"],
    "Angola": ["pt"],
    "Albânia": ["sq"],
    "Alemanha": ["de"],
    "Bélgica": ["nl", "fr", "de"],
    "Chade": ["fr", "ar"],
    "Uruguai": ["es"],
    "Azerbaijão": ["az"],
    "Bangladesh": ["bn"],
    "Barbados": ["en"],
    "Arábia Saudita": ["ar"],
    "Argélia": ["ar"],
    "Bahrein": ["ar"],
    "Belarus": ["be", "ru"],
    "Bolívia": ["es"],
    "Bósnia e Herzegovina": ["bs", "hr", "sr"],
    "Botsuana": ["en"],
    "Bulgária": ["bg"],
    "Argentina": ["es"],
    "Zimbábue": ["en"],
    "Quênia": ["sw", "en"],
    "Armênia": ["hy"],
    "Austrália": ["en"],
    "Burkina Faso": ["fr"],
    "Etiópia": ["am"],
    "Lituânia": ["lt"],
    "Macedônia do Norte": ["mk"],
    "Mônaco": ["fr"],
    "Marrocos": ["ar"],
    "Madagascar": ["mg", "fr"],
    "Palestina": ["ar"],
    "Burkina Faso": ["fr"],
    "Burundi": ["fr", "rn"],
    "Cabo Verde": ["pt"],
    "Cameroun": ["fr", "en"],
    "Canadá": ["en", "fr"],
    "Catar": ["ar"],
    "Cazaquistão": ["kk", "ru"],
    "República do Chade": ["fr", "ar"],
    "Chile": ["es"],
    "China": ["zh"],
    "Chipre": ["el", "tr"],
    "Colômbia": ["es"],
    "Congo": ["fr"],
    "Coreia do Norte": ["ko"],
    "Coreia do Sul": ["ko"],
    "Costa Rica": ["es"],
    "Côte d": ["fr"],
    "Croácia": ["hr"],
    "Cuba": ["es"],
    "Dinamarca": ["da"],
    "Djibouti": ["fr", "ar"],
    "Egito": ["ar"],
    "El Salvador": ["es"],
    "Emirados Árabes Unidos": ["ar"],
    "Equador": ["es"],
    "Eslováquia": ["sk"],
    "Eslovênia": ["sl"],
    "Espanha": ["es"],
    "Estados Unidos": ["en"],
    "Filipinas": ["fil", "en"],
    "Finlândia": ["fi", "sv"],
    "França": ["fr"],
    "Gabão": ["fr"],
    "Gâmbia": ["en"],
    "Gana": ["en"],
    "Geórgia": ["ka"],
    "Granada": ["en"],
    "Grécia": ["el"],
    "Guatemala": ["es"],
    "Guiana": ["en"],
    "Guiné": ["pt"],
    "Guiné-Bissau": ["pt"],
    "Guiné Equatorial": ["pt", "es", "fr"],
    "Haiti": ["ht", "fr"],
    "Honduras": ["es"],
    "Hungria": ["hu"],
    "Iêmen": ["ar"],
    "Ilhas Marshall": ["en", "mh"],
    "Índia": ["hi", "en"],
    "Indonésia": ["id"],
    "Irã": ["fa"],
    "Iraque": ["ar", "ku"],
    "Irlanda": ["en", "ga"],
    "Islândia": ["is"],
    "Israel": ["he", "ar"],
    "Itália": ["it"],
    "Jamaica": ["en"],
    "Japão": ["ja"],
    "Jordânia": ["ar"],
    "Kuwait": ["ar"],
    "Lesoto": ["en", "st"],
    "Letônia": ["lv"],
    "Líbano": ["ar"],
    "Líbia": ["ar"],
    "Luxemburgo": ["lb", "fr", "de"],
    "Malásia": ["ms"],
    "Malawi": ["ny", "en"],
    "Mali": ["fr"],
    "Malta": ["mt", "en"],
    "Maurício": ["mfe", "fr", "en"],
    "Mauritânia": ["ar"],
    "México": ["es"],
    "Moçambique": ["pt"],
    "Moldova": ["ro"],
    "Mongólia": ["mn"],
    "Myanmar": ["my"],
    "Namíbia": ["en", "af"],
    "Nepal": ["ne"],
    "Nicarágua": ["es"],
    "Nigéria": ["en"],
    "Noruega": ["no", "nb", "nn"],
    "Nova Zelândia": ["mi", "en"],
    "Omã": ["ar"],
    "Ordem de Malta": ["it", "en", "fr", "de", "es"],
    "Países Baixos": ["nl"],
    "Panamá": ["es"],
    "Paraguai": ["es", "gn"],
    "Peru": ["es"],
    "Polônia": ["pl"],
    "Portugal": ["pt"],
    "Reino Unido": ["en"],
    "República Centro-Africana": ["fr", "sg"],
    "República Dominicana": ["es"],
    "República Tcheca": ["cs"],
    "Romênia": ["ro"],
    "Ruanda": ["rw", "fr", "en"],
    "Rússia": ["ru"],
    "San Marino": ["it"],
    "Seicheles": ["fr", "en"],
    "Senegal": ["fr"],
    "Serra Leoa": ["en"],
    "Sérvia": ["sr"],
    "Singapura": ["en", "ms", "ta", "zh"],
    "Síria": ["ar"],
    "Sri Lanka": ["si", "ta"],
    "Suazilândia": ["en", "ss"],
    "Sudão": ["ar", "en"],
    "Suécia": ["sv"],
    "Suíça": ["de", "fr", "it", "rm"],
    "Suriname": ["nl"],
    "Tailândia": ["th"],
    "Tanzânia": ["sw", "en"],
    "Timor-Leste": ["pt", "tet"],
    "Togo": ["fr"],
    "Trinidad e Tobago": ["en"],
    "Tunísia": ["ar"],
    "Turquia": ["tr"],
    "Ucrânia": ["uk"],
    "Uganda": ["sw", "en"],
    "Uruguai": ["es"],
    "Uzbequistão": ["uz", "ka"],
    "Venezuela": ["es"],
    "Vietnã": ["vi"],
    "Zâmbia": ["en"],
}

### Methods

In [15]:
BODY = str("""
Prezados Senhores/Senhoras,
Espero que esta mensagem os encontre bem. É com grande respeito e admiração que me dirijo a Vossas Excelências, representantes da honrosa <EMBAIXADA>.
Permitam-me expressar, em primeiro lugar, minha imensa paixão e interesse pela rica e diversificada cultura de <PAIS>. Desde muito jovem, nutro um profundo fascínio pelas tradições, história, gastronomia e arte deste país encantador. Acredito que a cultura é uma ponte que conecta nações e pessoas, e minha dedicação em aprender mais sobre o seu país tem enriquecido minha vida de maneira singular.
Conhecer e compreender diferentes culturas é uma experiência enriquecedora, capaz de expandir horizontes e promover o entendimento mútuo. Com isso em mente, venho, humildemente, solicitar se seria possível obter um media kit, bandeiras, broches ou qualquer souvenir que Vossas Excelências tenham a disposição, de forma a materializar minha admiração e carinho pela cultura de <PAIS>.
Ter um item autêntico enviado diretamente de uma repartição diplomática seria uma honra inigualável para mim, e eu cuidaria deste tesouro com o mais profundo respeito e apreço.
Reitero meu compromisso em preservar a tradição de seu país e divulgar, sempre que possível, os valores e belezas culturais que ele representa. Acredito que o intercâmbio cultural é uma via de mão dupla, e farei questão de promover a amizade entre nossos povos através dessa maravilhosa conexão.
Desde já, agradeço pela atenção dispensada e, caso haja a possibilidade de atender a minha solicitação, ficarei imensamente grato.
Aproveito a oportunidade para expressar meus votos de prosperidade e bem-estar a Vossas Excelências e toda equipe da <EMBAIXADA>.
Atenciosamente,

Thiago Casa Nova da Luz

<ADDRESS>

P.S.: Essa mensagem foi traduzida automaticamente para os demais idiomas que possam ser utilizados. Peço desculpas por quaisquer erros ou traduções indevidas.
""")

In [48]:
def translate_text(text, target_language, source_language='pt'):
    """
    Translates a text from the source language to the target language.

    Args:
        text (str): The text to be translated.
        target_language (str): The target language code (e.g., 'fr' for French, 'es' for Spanish).
        source_language (str): The source language code (e.g., 'en' for English, 'auto' to auto-detect).

    Returns:
        str: The translated text in the target language.
    """
    print(f"Translating from PT to {target_language}")
    try:
        translator = Translator()
        translated = translator.translate(text, src=source_language, dest=target_language)
        return translated.text
    except:
        print("Cannot translate: ", target_language)
        return None

In [18]:
def prepare_body(body: str, country: list, subject: bool = False) -> str:
    if subject is False:
        final_body = body
        for language in country:
            translated_body = translate_text(body, language)
            if translated_body is not None:
                final_body = str(f"""
                {final_body}\n\r
                -----------{language}-----------\n\r
                {translated_body}
                """)
        return final_body
    else:
        final_subject = body
        for language in country:
            translated_subject = translate_text(body, language)
            if translated_subject is not None:
                final_subject= str(f"{final_subject} - {translated_subject}")
        return final_subject
        

In [20]:
import re

def extract_country(text):
    countries = [
        "Áustria", "África do Sul", "Afeganistão", "Angola", "Albânia", "Alemanha", "Bélgica",
        "Chade", "Uruguai", "Azerbaijão", "Bangladesh", "Barbados", "Arábia Saudita", "Argélia",
        "Bahrein", "Belarus", "Bolívia", "Bósnia e Herzegovina", "Botsuana", "Bulgária", "Argentina",
        "Zimbábue", "Quênia", "Armênia", "Austrália", "Burkina Faso", "Etiópia", "Lituânia",
        "Macedônia do Norte", "Mônaco", "Marrocos", "Madagascar", "Palestina",
        "Burkina Faso", "Burundi", "Cabo Verde", "Cameroun", "Canadá", "Catar", "Cazaquistão",
        "República do Chade", "Chile", "China", "Chipre", "Colômbia", "Congo", "Coreia do Norte",
        "Coreia do Sul", "Costa Rica", "Côte d", "Croácia", "Cuba", "Dinamarca", "Djibouti",
        "Egito", "El Salvador", "Emirados Árabes Unidos", "Equador", "Eslováquia", "Eslovênia",
        "Espanha", "Estados Unidos", "Filipinas", "Finlândia", "França", "Gabão", "Gâmbia",
        "Gana", "Geórgia", "Granada", "Grécia", "Guatemala", "Guiana", "Guiné", "Guiné-Bissau",
        "Guiné Equatorial", "Haiti", "Honduras", "Hungria", "Iêmen", "Ilhas Marshall", "Índia",
        "Indonésia", "Irã", "Iraque", "Irlanda", "Islândia", "Israel", "Itália", "Jamaica",
        "Japão", "Jordânia", "Kuwait", "Lesoto", "Letônia", "Líbano", "Líbia", "Luxemburgo",
        "Malásia", "Malawi", "Mali", "Malta", "Maurício", "Mauritânia", "México", "Moçambique",
        "Moldova", "Mongólia", "Myanmar", "Namíbia", "Nepal", "Nicarágua", "Nigéria", "Noruega",
        "Nova Zelândia", "Omã", "Ordem de Malta", "Países Baixos", "Panamá", "Paraguai", "Peru",
        "Polônia", "Portugal", "Reino Unido", "República Centro-Africana", "República Dominicana",
        "República Tcheca", "Romênia", "Ruanda", "Rússia", "San Marino", "Seicheles", "Senegal",
        "Serra Leoa", "Sérvia", "Singapura", "Síria", "Sri Lanka", "Suazilândia", "Sudão", "Suécia",
        "Suíça", "Suriname", "Tailândia", "Tanzânia", "Timor-Leste", "Togo", "Trinidad e Tobago",
        "Tunísia", "Turquia", "Ucrânia", "Uganda", "Uruguai", "Uzbequistão", "Venezuela", "Vietnã",
        "Zâmbia"
    ]
    pattern = r'\b(?:' + '|'.join(countries) + r')\b'
    result = re.search(pattern, text, re.IGNORECASE)
    return result.group() if result else None

In [45]:
try_again_list = dict()

In [ ]:
SCOPES = [
    "https://www.googleapis.com/auth/gmail.send"
]
flow = InstalledAppFlow.from_client_secrets_file('not_credentials.json', SCOPES)
creds = flow.run_local_server(port=0)
service = build('gmail', 'v1', credentials=creds)


def generate_email(embassy: str, emails: list, country: str):
    body = prepare_body(BODY.replace("<PAIS>", country).replace("<EMBAIXADA>", embassy), country_code[country])
    subject = prepare_body("Pedido de Colecionador", country_code[country], subject=True)
    message = MIMEText(body)
    message['to'] = ", ".join(emails)
    message['subject'] = subject
    create_message = {'raw': base64.urlsafe_b64encode(message.as_bytes()).decode()}
    try:
        message = (service.users().messages().send(userId="me", body=create_message).execute())
        print(F'sent message to {embassy} emails: {", ".join(emails)}')
    except Exception as error:
        print(F'An error occurred: {error}')
        print(F'No email to {embassy} was sent!!!')
        try_again_list[embassy] = emails
        message = None


### Mailing loop to send email to each Embassy

In [47]:
idx = 566 #used only if some error happen so I can start from the last successful index
for embassy, emails in tqdm(emails_dict.items(), desc="Enviando Emails:"):
    idx += 1
    if len(emails) and idx > 10:
        generate_email(embassy, emails, extract_country(embassy))

Enviando Emails::   0%|          | 0/566 [00:00<?, ?it/s]

Translating from PT to es
Translating from PT to es
An error occurred: <HttpError 400 when requesting https://gmail.googleapis.com/gmail/v1/users/me/messages/send?alt=json returned "Invalid To header". Details: "[{'message': 'Invalid To header', 'domain': 'global', 'reason': 'invalidArgument'}]">
No email to Embaixada do Peru em Brasília was sent!!!
Translating from PT to es
Translating from PT to es
sent message to Consulado-Geral do Peru em Manaus emails: conpemao@gmail.com
Translating from PT to es
Translating from PT to es
sent message to Consulado-Geral do Peru em Rio Branco emails: consulperu-riobranco@rree.gob.xn--9ca, consuladoperuac@gmail.com
Translating from PT to es
Translating from PT to es
sent message to Consulado-Geral do Peru no Rio de Janeiro emails: consulado@consuladoperurio.com.br
Translating from PT to es
Translating from PT to es
sent message to Consulado Honorário do Peru em Curitiba emails: consuladoperu.parana.gov@hotmail.com
Translating from PT to es
Translati